In [1]:
import pandas as pd
from math import floor, ceil
import math
import numpy as np
from collections import Counter
import os
from nltk import corpus
import string
import itertools
import boto3
import re
import io
import string
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from matplotlib import pyplot as plt
import re

In [2]:
AWS_S3_BUCKET = 'esgnie'

def get_s3_companies(s3_client, bucket='esgnie', comp_prefix='companies/'):
    ## get companies
    result = s3_client.list_objects(Bucket=bucket, Prefix=comp_prefix, Delimiter='/')
    l_subdir = [com_prefix.get('Prefix').replace(comp_prefix, '').replace('/', '')
                for com_prefix in result.get('CommonPrefixes')]
    return l_subdir

def get_company_files(s3_client, ticker, bucket='esgnie', filename_pattern=None, comp_prefix='companies'):
    result = s3_client.list_objects(Bucket=bucket, Prefix=f"{comp_prefix}/{ticker}/")
    df_files = pd.DataFrame()
    df_files['file'] = [obj['Key'] for obj in result['Contents']]
    df_files['filename'] = [file.split('/')[-1] for file in df_files['file']]
    if filename_pattern is not None:
        df_files['pattern_match'] = [1 if re.match(filename_pattern, file) else 0 for file in df_files['filename']]
    # df_files[df_files['pattern_match'] == 1]['filename'].tolist()
    return df_files[df_files['pattern_match'] == 1]['file'].tolist()

def write_csv_to_s3(s3_client, df, file, bucket='esgnie'):
    with io.StringIO() as csv_buffer:
        df.to_csv(csv_buffer, index=False)
        response = s3_client.put_object(
            Bucket=bucket, Key=file, Body=csv_buffer.getvalue()
        )
        status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")
    return status


In [3]:
s3_client = boto3.client('s3')
l_ticker = get_s3_companies(s3_client)

In [20]:
def quants_parser(df):
    '''
    input: df_cell file 
    output: header_rows, non_quant_rows, quant_rows, quant_rows information 
    
    '''
    
    
    
    df = df.drop(['Unnamed: 0', "file", 'level_3', 'xbar','ybar', 
                  'upright', 'direction','digits_ratio', 'years_ratio',
                  'dates_ratio', 'keyword_flag','keyword_col_flag','keyword_row_flag'], axis=1)
    
    df["text_height"] = df.bottom-df.top

    is_numeric = []
    for text in list(df.text):  
        t="".join(c for c in text if c.isalnum())
        is_numeric.append(t.isnumeric())

    df["numeric_flag"] = is_numeric
    
        ###masks
    quantmask1 = df["quant_row_flag"] == 1
    yearmask1 = df["year_flag"] == 1
    
    if len(df.text_height.mode()) == 1:
        heightmask = df["text_height"] > float(df.text_height.mode())
    else: 
        heightmask = df["text_height"] > float(df.text_height.min())
        
    textmask = df["numeric_flag"] == 0
    
    header_rows = df[heightmask & ~quantmask1 & textmask]
    items = df[quantmask1 & ~yearmask1 & ~heightmask & textmask]
    nonquantrows = pd.concat([items, df]).drop_duplicates(keep=False)
    
    mask1 = items["text"] != "Indicator"
    mask2 = items["text"] != "Items"

    quant_rows = items[mask1 & mask2].row_num.unique()
    quantmask2 = df["row_num"].isin(quant_rows)

    quant_rows = df[quantmask2 & ~textmask]

    rows_of_df = []
    columns_of_df = ["Company", "File Name", "Page Number", "Variable", "Date", "Value"]


    def f(qr):
        left_coord = qr.x0
        right_coord = qr.x1
        leftmask = df.x0  >= math.floor(qr.x0) -25 
        rightmask = df.x1 <= math.ceil(qr.x1) + 25
        rowmask2 = df.row_num <= qr.row_num
        quantmask2 = df.row_num.isin(quant_rows)
        
        company = df.company.iloc[0]
        filename = df.filename.iloc[0]
        pagenum = df.pagenum.iloc[0]
        
        if items[items.row_num == qr.row_num].empty:
            variable = " "
        else:
            variable = items[items.row_num == qr.row_num].iloc[0].text
        
        if df[~quantmask2 & leftmask & rightmask & rowmask2 & yearmask1].empty:
            date = " "
        else:
            date = df[~quantmask2 & leftmask & rightmask & rowmask2 & yearmask1].iloc[0].text
        
        value = qr.text

        row = [company, filename, pagenum, variable, date, value]
        rows_of_df.append(row)
        return rows_of_df

    quant_rows.apply(f,axis = 1)

    df2 = pd.DataFrame(rows_of_df, columns = columns_of_df)
    #df2 = df2[df2["Date"] != " "]
    
    return header_rows, nonquantrows, items, df2

In [5]:
def textfinder(items):
    L = list(items.text)

    words = ' '.join(L).lower()
    words = words.translate(words.maketrans('', '', string.digits))

    L = words.split(" ")
    total_num_of_words = len(L)


    dic = Counter(L)
    sortedtext = {k: v for k, v in sorted(dic.items(), key=lambda item: item[1], reverse = True)}
    proportion = np.array(list(sortedtext.values()))/total_num_of_words
    
    if items.company.empty:
        ticker = []
    else:
        ticker = list(items.company)[0] * np.ones(len(proportion))
    
    
    
    df = pd.DataFrame.from_dict({"company": ticker, "word": sortedtext.keys(),
                                 "freq": sortedtext.values(), "proportion": proportion})
    
    return words, df

In [6]:
def commonwords(final):
    custom = ["", "must", "co.", "ltd.", "()", "')", "co.,", "(cid:)", ",,.", ",,,.", "year", "rmb", "december", "total", "company",
         "china", "group", "others", "two", "within", "one", "item", "million", "billion", "sinopec", "–", "—", ",,", "(,)"]
    stopwords = corpus.stopwords.words('english')
    
    stopwords = stopwords + list(string.punctuation) + list(string.digits) + custom
    words, df = textfinder(final)
    sw = df.word.isin(stopwords)
    
    #wordcloud = WordCloud(stopwords=stopwords, background_color="white", height = 300, max_words = 100).generate(words)
    #plt.imshow(wordcloud, interpolation='bilinear')
    #plt.axis("off")
    #plt.show()
    
    return df[~sw]

In [24]:
def parse_by_ticker(ticker, groupby_variables = None, groupby_values = None, word_freq = False):
    '''
    input: company ticker
    output: everything
    
    '''
    
    l_files = get_company_files(s3_client, ticker=ticker, filename_pattern='^df_cells')
    
    print(len(l_files))
    finalDF = pd.DataFrame()
    finalITEMS = pd.DataFrame()
    finalNONQUANT = pd.DataFrame()
    
    for file in l_files:
        obj = s3_client.get_object(Bucket='esgnie', Key=file)
        df = pd.read_csv(obj['Body'])
        
        if groupby_variables != None:
            for i in range(len(groupby_variables)):
                df = df[df[groupby_variables[i]] == groupby_values[i]]
            
            
        header_rows, nonquantrows, items, df2 = quants_parser(df)
        finalDF = finalDF.append(df2)
        finalITEMS = finalITEMS.append(items)
        finalNONQUANT = finalNONQUANT.append(nonquantrows)
    
    if word_freq == True:
    
        items = commonwords(finalITEMS)
        nonquant = commonwords(finalNONQUANT)
        overall = items.rename(columns={"freq": "freq_quant_row", "proportion": "proportion_quant_row"})


        freq_nonquant_row = []
        proportion_nonquant_row = []

        for word in items.word:
            if word in list(nonquant.word):
                freq_nonquant_row.append(int(nonquant[nonquant.word == word].freq))
                proportion_nonquant_row.append(float(nonquant[nonquant.word == word].proportion))
            else:
                freq_nonquant_row.append(0)
                proportion_nonquant_row.append(0)

        overall["freq_nonquant_row"] = freq_nonquant_row
        overall["proportion_nonquant_row"] = proportion_nonquant_row
        overall["freq_diff"] = np.array(overall.freq_quant_row) - np.array(freq_nonquant_row)
        overall["proportion_diff"] = np.array(overall.proportion_quant_row) - np.array(proportion_nonquant_row)

        overall.sort_values(by=['proportion_diff'], ascending = False, inplace = True)
        
        status = write_csv_to_s3(s3_client, overall, "word_freq/%s_overall.csv"%ticker, bucket='esgnie')
        
        if status == 200:
            print("Successful saving word frequencies for %s"%(ticker))
        else:
            print("Unsuccessful saving word frequencies for %s"%(ticker))
            
        
    else:
        items, nonquant, overall = None, None, None
        
    
    #save files
    status = write_csv_to_s3(s3_client, finalDF,"tables/%s.csv" % ticker, bucket='esgnie')
    if status == 200:
        print("Successful saving tables for %s"%(ticker))
    else:
        print("Unsuccessful saving tables for %s"%(ticker))
        
        
    
    
    return finalDF, finalITEMS, finalNONQUANT, items, nonquant, overall
  

In [8]:
def wordfreq(l_ticker, groupby_variables = None, groupby_values = None):
    '''
    inputs:
    l_ticker: list of companies for which to parse for word frequencies
    
    
    '''
    
    overall_words = pd.DataFrame()
    
    for ticker in l_ticker:
        finalDF, finalITEMS, finalNONQUANT, items, nonquant, overall = parse_by_ticker(ticker, 
                                                                                       groupby_variables = groupby_variables,
                                                                                       groupby_values = groupby_values,
                                                                                      word_freq = True)
        overall_words.append(overall)
        
    status = write_csv_to_s3(s3_client, overall_words, "word_freq/%s_overall.csv"%l_ticker, bucket='esgnie')
        
    if status == 200:
        print("Successful saving word frequencies for %s"%(l_ticker))
    else:
        print("Unsuccessful saving word frequencies for %s"%(l_ticker))

  
    return overall_words
    

# Actual parsing

In [11]:
ticker = l_ticker[0]
finalDF, finalITEMS, finalNONQUANT, items, nonquant, overall = parse_by_ticker(ticker, groupby_variables = None, groupby_values = None, word_freq = True)

Successful saving word frequencies
Successful saving tables


In [38]:
for ticker in l_ticker[2:]:
    try:
        finalDF, finalITEMS, finalNONQUANT, items, nonquant, overall = parse_by_ticker(ticker, groupby_variables = None, groupby_values = None, word_freq = True)
    except (RuntimeError, TypeError, NameError, ValueError, ZeroDivisionError):
        continue
        

Successful saving word frequencies
Successful saving tables for 601288
Successful saving word frequencies
Successful saving tables for 601688
Successful saving word frequencies
Successful saving tables for 601766


In [ ]:
#wordfreq(l_ticker[:2], groupby_variables = ["pagenum"], groupby_values = [20])

In [14]:
finalDF, finalITEMS, finalNONQUANT, items, nonquant, overall = parse_by_ticker('601857', groupby_variables = None, groupby_values = None, word_freq = True)

Successful saving word frequencies for 601857
Successful saving tables for 601857


In [23]:
for ticker in l_ticker[-4:]:
    try:
        finalDF, finalITEMS, finalNONQUANT, items, nonquant, overall = parse_by_ticker(ticker, groupby_variables = None, groupby_values = None, word_freq = True)
    except (RuntimeError, TypeError, NameError, ValueError, ZeroDivisionError, AttributeError):
        continue

### Debugging

In [16]:
vale = pd.read_csv(r"F:\Downloads\df_cells_2011-sustainability-report_pagenum-51.csv")

In [29]:
l_ticker[-4]

'VALE3'

In [30]:
l_files = get_company_files(s3_client, ticker=l_ticker[-4], filename_pattern='^df_cells')

In [31]:
len(l_files)

0